In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import random
from sklearn.metrics import confusion_matrix
from gensim.models import word2vec
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.exceptions import ConvergenceWarning
import warnings
import seaborn as sns
import plotly as plt

In [5]:
df = pd.read_pickle("../data/nsfw_cleaned.pkl")

In [6]:
df.head()

,body,numberOfComments,over18,upVotes
0,didnt happen like year ago feel like fever dre...,95,False,392
1,f alway say 's ugli old stuff like nobodi like...,244,True,867
2,hi im f dental assist clean accident get prick...,265,False,920
3,context wife f parent beauti toddler ym f rent...,138,True,1372
4,yesterday 's fuckup spill go crazi f frequent ...,31,False,45


In [7]:
df.isnull().sum()

body                0
numberOfComments    0
over18              0
upVotes             0
dtype: int64

In [8]:
# split into train and test subsets
split = StratifiedShuffleSplit(n_splits=5, test_size=0.25)

for train_index, test_index in split.split(df,
                                           df["over18"]): 
    df_train = df.reindex(train_index)
    df_test = df.reindex(test_index)

In [9]:
df_train = df_train.drop(columns=["numberOfComments", "upVotes"])

In [10]:
df_train

,body,over18
578,ive see guy almost month like love first sight...,False
264,run home gym realli close hous woman stand aro...,False
359,get home univers hungri alreadi thought want m...,False
305,ampxb hi plea vote favorit post decemb post ac...,False
299,autisticdiagnos adult social cue import tend o...,False
...,...,...
303,day ago around walk back home grab smoke live ...,True
458,obligatori obligatori famili want uncl christm...,False
826,earlier post realiz want abstain sex bf month ...,True
300,ill prefac say good day overal last straw ish ...,False


In [11]:
len(df_train), len(df_test)

(665, 222)

In [12]:
df_train["over18"].value_counts()/len(df_train)

False    0.822556
True     0.177444
Name: over18, dtype: float64

In [13]:
df_test["over18"].value_counts()/len(df_test)

False    0.81982
True     0.18018
Name: over18, dtype: float64

In [54]:
models_params = [
    {'name': 'SVC', 'classifier': SVC(), 'param_grid': {'classifier__C': [0.1, 1, 10], 'classifier__kernel': ['linear', 'rbf']}},
    {'name': 'Logistic Regression', 'classifier': LogisticRegression(class_weight='balanced'), 'param_grid': {'classifier__C': [1], 'classifier__solver': ['saga']}},
    {'name': 'Logistic Regression_L2', 'classifier': LogisticRegression(class_weight='balanced'), 'param_grid': {'classifier__C': [1], 'classifier__penalty':['l2'], 'classifier__solver': ['saga']}},
    {'name': 'Random Forest', 'classifier': RandomForestClassifier(class_weight='balanced'), 'param_grid': {'classifier__n_estimators': [50, 100], 'classifier__max_depth': [5, 10, 20]}}
]

In [33]:
for model_params in models_params:
    random.seed(20)
    # Create a pipeline for TF-IDF Vectorization and model training
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('classifier', model_params['classifier'])
    ])

    # Combine parameter grid from models_params with TF-IDF specific parameters
    param_grid = {
        'tfidf__max_features': [1000, 5000, None],
        'tfidf__ngram_range': [(1, 1), (1, 2)],
        **model_params['param_grid']
    }

    # Perform grid search with F1 score as the scoring metric
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', verbose=1)
    grid_search.fit(df_train['body'], df_train["over18"])

    # Get best model
    best_model = grid_search.best_estimator_

    # Predict on test data
    predictions = best_model.predict(df_test['body'])
    conf_matrix = confusion_matrix(df_test["over18"], predictions)
    
    # Print F1 score on validation (grid_search) point
    print(f"{model_params['name']} - Best F1 Score on Validation: {grid_search.best_score_}")

    # Print confusion matrix
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f"Confusion Matrix: {model_params['name']}" )
    plt.show()
    
    # Calculate F1 score on test data
    f1_test = f1_score(df_test["over18"], predictions)
    print(f"{model_params['name']} - F1 Score on Test: {f1_test}\n")

Fitting 5 folds for each of 36 candidates, totalling 180 fits
SVC - Best F1 Score on Validation: 0.3327400588026775

Confusion Matrix (SVC):
 [[170  12]
 [ 30  10]]
SVC - F1 Score on Test: 0.3225806451612903

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Logistic Regression - Best F1 Score on Validation: 0.4540926916221034

Confusion Matrix (Logistic Regression):
 [[154  28]
 [ 28  12]]
Logistic Regression - F1 Score on Test: 0.3

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Logistic Regression_L2 - Best F1 Score on Validation: 0.4540926916221034

Confusion Matrix (Logistic Regression_L2):
 [[154  28]
 [ 28  12]]
Logistic Regression_L2 - F1 Score on Test: 0.3

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Random Forest - Best F1 Score on Validation: 0.18933735897028442

Confusion Matrix (Random Forest):
 [[179   3]
 [ 38   2]]
Random Forest - F1 Score on Test: 0.0888888888888889



In [46]:
# Tokenize sentences
tokenized_train_sentences = [sentence.split() for sentence in df_train['body']]
tokenized_test_sentences = [sentence.split() for sentence in df_test['body']]

# Train Word2Vec model
word2vec_model = word2vec.Word2Vec(sentences=tokenized_train_sentences, vector_size=100, window=5, min_count=1, workers=10)

# Transform tokenized sentences using Word2Vec
X_train_embedded = np.array([np.mean([word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv] or [np.zeros(word2vec_model.vector_size)], axis=0) for sentence in tokenized_train_sentences])
X_test_embedded = np.array([np.mean([word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv] or [np.zeros(word2vec_model.vector_size)], axis=0) for sentence in tokenized_test_sentences])

In [52]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [55]:

for model_params in models_params:
    # Create a pipeline for clustering and model training
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Standardize the features
        ('pca', PCA(n_components=100)),  # Principal Component Analysis for dimensionality reduction
        ('classifier', model_params['classifier'])
    ])

    # Combine parameter grid from models_params with embedding specific parameters
    param_grid = {
        **model_params['param_grid']
    }

    # Perform grid search with F1 score as the scoring metric
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', verbose=1)
    grid_search.fit(X_train_embedded, df_train["over18"])

    # Get best model
    best_model = grid_search.best_estimator_

    # Predict on test data
    predictions = best_model.predict(X_test_embedded)
    conf_matrix = confusion_matrix(df_test["over18"], predictions)
    
    # Print F1 score on validation (grid_search) point
    print(f"{model_params['name']} - Best F1 Score on Validation: {grid_search.best_score_}")

    # Print confusion matrix
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f"Confusion Matrix: {model_params['name']}" )
    plt.show()
    
    # Calculate F1 score on test data
    f1_test = f1_score(df_test["over18"], predictions)
    print(f"{model_params['name']} - F1 Score on Test: {f1_test}\n")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
SVC - Best F1 Score on Validation: 0.0

Confusion Matrix (SVC):
 [[182   0]
 [ 40   0]]
SVC - F1 Score on Test: 0.0

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Logistic Regression - Best F1 Score on Validation: 0.3200333274505549

Confusion Matrix (Logistic Regression):
 [[117  65]
 [ 24  16]]
Logistic Regression - F1 Score on Test: 0.2644628099173554

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Logistic Regression_L2 - Best F1 Score on Validation: 0.3193012295324583

Confusion Matrix (Logistic Regression_L2):
 [[117  65]
 [ 24  16]]
Logistic Regression_L2 - F1 Score on Test: 0.2644628099173554

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Random Forest - Best F1 Score on Validation: 0.04444444444444444

Confusion Matrix (Random Forest):
 [[181   1]
 [ 37   3]]
Random Forest - F1 Score on Test: 0.13636363636363635

